In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [30]:
data_type = 'N1' 
TS_Fresh_setting = 'Efficient'

In [31]:
#Change load path to the band power time series folder
load_path = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
load_path_bpw = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
load_path_max_freqs = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'

X_bpw = pd.read_hdf(load_path_bpw + data_type +  'six_second_freq_df.h5', key='df', mode='r')
y = pd.read_hdf(load_path + data_type +  '_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + data_type +  '_groups.h5', key='df', mode='r')
X_bpw, y , groups = X_bpw.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)

X_max_freqs = pd.read_hdf(load_path_max_freqs + data_type +  'six_second_max_freq_stats_df.h5', key='df', mode='r')
X_max_freqs = X_max_freqs.reset_index(drop = True)

X = pd.concat([X_bpw , X_max_freqs], axis = 1)

#Transform the X into TS_Fresh Features___
# 1. Convert to TS_Fresh format Dataframe 
ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(X, ts_cols = list(X.columns))

# 2. Extract TS_Fresh Features from the dataframe
if TS_Fresh_setting == 'Minimal':
    settings = MinimalFCParameters()
elif TS_Fresh_setting == 'Efficient':
    settings = EfficientFCParameters()
else:
    raise Exception('No TS Fresh Parameter Setting Set!!')

extracted_ts_fresh_df = extract_features(ts_fresh_df, column_id = 'id' , column_sort = 'time',  default_fc_parameters=settings)



# 3. Asign extract_ts_fresh_df to the variable X
X = extracted_ts_fresh_df.copy()
print(len(X.columns))
#Drop columns where all values are NA 
X = X.dropna(axis = 1)
print(len(X.columns))

#Drop columns where all values are the same
# Find columns where all values are the same
same_value_columns = X.columns[X.nunique() == 1]
# Drop columns with the same values
X = X.drop(columns=same_value_columns)
print(len(X.columns))


#Generate a region to features dictionary - this will enable us to run expts regionally as before
regional_features_dict = {}
region_channel_dict = constants.region_to_channel_dict
regions = list(region_channel_dict.keys())

for region in regions:
    region_features = [col for col in X.columns if region + '_' in col]
    regional_features_dict[region] = region_features 


18-Nov-23 21:32:03 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [04:34<00:00,  2.28s/it]


151515
76570
65942


In [42]:
#Run the experiment with hyperparameter tuning
t1 = time.time()
clf_dict = {'DT' : AdaBoostClassifier(random_state = 5)}
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2)

results_df, clfs_dict = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt,clf_dict, return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2], groups_for_fit = False, best_params = False, return_clfs = True)

t2 = time.time()

print(str(t2-t1))

acc_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
acc_df.iloc[:, 1:].mean(axis = 0).max()

182.3664083480835


0.861111111111111

In [35]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [43]:
acc_df

,random_state,Prefrontal_DT_acc,Prefrontal_DT_std,Frontal_DT_acc,Frontal_DT_std,Left Frontal_DT_acc,Left Frontal_DT_std,Right Frontal_DT_acc,Right Frontal_DT_std,Central_DT_acc,Central_DT_std,Left Central_DT_acc,Left Central_DT_std,Right Central_DT_acc,Right Central_DT_std,Left Temporal_DT_acc,Left Temporal_DT_std,Right Temporal_DT_acc,Right Temporal_DT_std,Parietal_DT_acc,Parietal_DT_std,Left Parietal_DT_acc,Left Parietal_DT_std,Right Parietal_DT_acc,Right Parietal_DT_std,Occipital_DT_acc,Occipital_DT_std
0,1,0.508889,0.147137,0.571111,0.190853,0.704444,0.067987,0.448889,0.056917,0.531111,0.112962,0.806667,0.148341,0.588889,0.157762,0.511111,0.166147,0.555556,0.112437,0.451111,0.197715,0.513333,0.092803,0.466667,0.177639,0.577778,0.075359
1,2,0.573333,0.097017,0.580000,0.178941,0.620000,0.138136,0.526667,0.126803,0.722222,0.201966,0.915556,0.082132,0.597778,0.118509,0.517778,0.171500,0.660000,0.154536,0.662222,0.118759,0.597778,0.095426,0.677778,0.126491,0.448889,0.155492


In [41]:
# This is the ADA efficient, random_state = 10 data_frame, max acc = 80% , experiment 1
acc_df

,random_state,Prefrontal_DT_acc,Prefrontal_DT_std,Frontal_DT_acc,Frontal_DT_std,Left Frontal_DT_acc,Left Frontal_DT_std,Right Frontal_DT_acc,Right Frontal_DT_std,Central_DT_acc,Central_DT_std,Left Central_DT_acc,Left Central_DT_std,Right Central_DT_acc,Right Central_DT_std,Left Temporal_DT_acc,Left Temporal_DT_std,Right Temporal_DT_acc,Right Temporal_DT_std,Parietal_DT_acc,Parietal_DT_std,Left Parietal_DT_acc,Left Parietal_DT_std,Right Parietal_DT_acc,Right Parietal_DT_std,Occipital_DT_acc,Occipital_DT_std
0,1,0.65,0.093541,0.775,0.183712,0.75,0.176777,0.7,0.100000,0.775,0.145774,0.775,0.165831,0.750,0.136931,0.575,0.280624,0.775,0.122474,0.70,0.257391,0.725,0.145774,0.675,0.169558,0.60,0.242384
1,2,0.65,0.215058,0.750,0.223607,0.80,0.100000,0.6,0.122474,0.725,0.145774,0.800,0.169558,0.625,0.250000,0.600,0.122474,0.675,0.100000,0.65,0.165831,0.675,0.169558,0.625,0.176777,0.65,0.165831


In [38]:
# This is the ADA efficient, random_state = 5 data_frame, max acc = 80% 
ada_efficient_random_state_5_expt_1

,random_state,Prefrontal_DT_acc,Prefrontal_DT_std,Frontal_DT_acc,Frontal_DT_std,Left Frontal_DT_acc,Left Frontal_DT_std,Right Frontal_DT_acc,Right Frontal_DT_std,Central_DT_acc,Central_DT_std,Left Central_DT_acc,Left Central_DT_std,Right Central_DT_acc,Right Central_DT_std,Left Temporal_DT_acc,Left Temporal_DT_std,Right Temporal_DT_acc,Right Temporal_DT_std,Parietal_DT_acc,Parietal_DT_std,Left Parietal_DT_acc,Left Parietal_DT_std,Right Parietal_DT_acc,Right Parietal_DT_std,Occipital_DT_acc,Occipital_DT_std
0,1,0.675,0.061237,0.8,0.169558,0.775,0.165831,0.7,0.100000,0.775,0.145774,0.8,0.127475,0.75,0.136931,0.575,0.280624,0.775,0.122474,0.70,0.257391,0.725,0.145774,0.675,0.169558,0.60,0.242384
1,2,0.650,0.215058,0.7,0.187083,0.725,0.145774,0.6,0.183712,0.650,0.165831,0.7,0.150000,0.65,0.242384,0.600,0.122474,0.700,0.100000,0.65,0.165831,0.700,0.203101,0.600,0.200000,0.65,0.165831


0.825

In [11]:
acc_df.iloc[:, 1:].max(axis = 1)

0    0.80
1    0.85
dtype: float64